In [2]:
from pyspark.sql import functions as F
from IPython.display import HTML

# =========================================================
# 1) LER BRONZE
# =========================================================

df_bronze = spark.read.parquet("Files/Voos/")

print("=== AMOSTRA BRONZE ===")
#df_bronze.show(10, truncate=False)

# =========================================================
# 2) AJUSTE DAS COLUNAS (SILVER)
# =========================================================

df_silver = (
    df_bronze
        # Converter timestamps (UTC)
        .withColumn(
            "dt_partida_prevista_utc",
            F.to_timestamp("dt_partida_prevista_utc", "dd/MM/yyyy HH:mm")
        )
        .withColumn(
            "dt_chegada_prevista_utc",
            F.to_timestamp("dt_chegada_prevista_utc", "dd/MM/yyyy HH:mm")
        )
        .withColumn(
            "dt_referencia",
            F.to_date("dt_referencia", "dd/MM/yyyy")
        )

        # Remover espaços de strings
        .withColumn("nr_voo", F.trim("nr_voo"))
        .withColumn("nr_etapa", F.trim("nr_etapa"))
        .withColumn("sg_empresa_icao", F.trim("sg_empresa_icao"))
        .withColumn("sg_icao_destino", F.trim("sg_icao_destino"))
        .withColumn("sg_icao_origem", F.trim("sg_icao_origem"))
        .withColumn("ds_tipo_servico", F.trim("ds_tipo_servico"))
        .withColumn("tx_codeshare", F.trim("tx_codeshare"))
        .withColumn("sg_equipamento_icao", F.trim("sg_equipamento_icao"))

        # Converter numéricos
        .withColumn("qt_assentos_previstos",
            F.regexp_replace("qt_assentos_previstos", r"\D", "").cast("int")
        )
)

print("=== SILVER APÓS TRATAMENTO ===")
df_silver.show(10, truncate=False)




# =========================================================
# 3) SALVAR COMO DELTA
# =========================================================

df_silver.write.format("delta").mode("overwrite").saveAsTable("voos_silver")


# =========================================================
# 4) FUNÇÃO DE DISPLAY ESTILIZADO
# =========================================================

def display_silver(df_silver, n=20):
    pdf = df_silver.limit(n).toPandas()

    styled = (
        pdf.style
        .set_table_styles([
        
        ])
        .set_properties(**{"text-align": "left"})
    )

    return HTML(styled.to_html())


# =========================================================
# 5) EXIBIR 30 LINHAS DA SILVER
# =========================================================

html = display_silver(df_silver, 30)
#display(html)








StatementMeta(, c621c2c2-0ab8-4e4a-a9f3-8db2b592f90c, 4, Finished, Available, Finished)

=== AMOSTRA BRONZE ===
+------------------------------------+-----------------------+-----------------------+-------------+--------+------+---------------------+---------------+-------------------+---------------+--------------+----------------------------+-------------+
|ds_tipo_servico                     |dt_chegada_prevista_utc|dt_partida_prevista_utc|dt_referencia|nr_etapa|nr_voo|qt_assentos_previstos|sg_empresa_icao|sg_equipamento_icao|sg_icao_destino|sg_icao_origem|tx_codeshare                |data_consulta|
+------------------------------------+-----------------------+-----------------------+-------------+--------+------+---------------------+---------------+-------------------+---------------+--------------+----------------------------+-------------+
|REGULAR DE PASSAGEIROS INTERNACIONAL|05/01/2025 01:05       |04/01/2025 23:55       |04/01/2025   |1       |0006  |168                  |TAP            |A21N               |SBSG           |SBMO          |                         

In [ ]:
# Colunas a remover
cols_to_drop = ["nr_etapa", "nr_voo", "tx_codeshare", "data_consulta"]

# Cria um df temporário sem essas colunas (não grava nada)
df_silver_preview = df_silver.drop(*cols_to_drop)

# Mostrar schema e primeiras linhas para validação
print("=== Schema PREVIEW ===")
df_silver_preview.printSchema()
print("\n=== 10 linhas PREVIEW ===")
df_silver_preview.show(10, truncate=False)


In [4]:
# grava nova tabela (não sobrescreve a antiga)
df_silver_preview.write.format("delta").mode("overwrite").saveAsTable("voos_silver_v2")

print("✅ Nova tabela criada: voos_silver_v2")


StatementMeta(, c621c2c2-0ab8-4e4a-a9f3-8db2b592f90c, 6, Finished, Available, Finished)

✅ Nova tabela criada: voos_silver_v2
